In [1]:
from utils import get_reviews

texts = get_reviews()['review'].tolist()

In [2]:
from transformers.modeling_distilbert import DistilBertModel
from transformers.tokenization_distilbert import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
model = DistilBertModel.from_pretrained('distilbert-base-cased')

In [3]:
import numpy as np
import torch

def get_embedding(texts, tokenizer, model, max_len):
    input_ids = torch.LongTensor(
        tokenizer.batch_encode_plus(
        texts, max_length=max_len, pad_to_max_length=True
    )['input_ids'])
    
    return model(input_ids)[0].detach().numpy()

In [4]:
from tqdm import tqdm

max_len = 250
shape = (len(texts), max_len, model.config.dim)

out_of_memmory_array = np.memmap(
    f'bert_features_shape_{shape}.dat', dtype='float16', mode='w+',
    shape=shape)


batch_size = 64
n_batches = int(len(texts)/batch_size)

for batch_end_index in tqdm(range(batch_size,
                                  len(texts),
                                  batch_size),
                            total=n_batches):
    
    batch = texts[batch_end_index - batch_size: batch_end_index]
    embeddings = get_embedding(batch, tokenizer, model, max_len)
    out_of_memmory_array[batch_end_index - batch_size:batch_end_index, :, :] = embeddings

100%|██████████| 781/781 [1:22:02<00:00,  6.30s/it]
